### Read CSV with top 100 songs bilboards

In [ ]:
import pandas as pd

bilboard_df1 = pd.read_csv("../datas/top100.csv", low_memory=False)
bilboard_df2 = pd.read_csv("../datas/top100oct.csv", low_memory=False)
bilboard_df1


In [ ]:
bilboard_df2

##### Compare 2 DF and print the duplicate items

In [ ]:
def duplicate_bilboards():
    for song, songs in enumerate(bilboard_df1['title']):
        for sound, sounds in enumerate(bilboard_df2['title']):
            if songs == sounds:
                playlist = {
                    "Musica": bilboard_df1['title'][sound],
                    "Artista": bilboard_df2['artist'][sound]
                }
                print(playlist)
duplicate_bilboards()

##### SparkSession create and Read csv

In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()
path = "../datas/top100oct.csv"
path2 = "../datas/top100.csv"
bilboard_df1_spark = spark.read.csv(path)
bilboard_df1_spark

##### Get all data with Schema context

In [ ]:
bilboard_df1_spark = spark.read.format("csv").option("inferSchema", "True").option("header", "True").csv(path)
bilboard_df1_spark.collect()

In [ ]:
bilboard_df2_spark = spark.read.format("csv").option("inferSchema", "True").option("header", "True").csv(path2)
bilboard_df2_spark.collect()

##### Show schema

In [ ]:
bilboard_df1_spark.printSchema()

In [294]:
bilboard_df2_spark.printSchema()

root
 |-- title: string (nullable = true)
 |-- artist: string (nullable = true)
 |-- featured_artist: string (nullable = true)
 |-- this_week_chart: integer (nullable = true)
 |-- last_week_chart: string (nullable = true)
 |-- peak: integer (nullable = true)
 |-- weeks_on_chart: integer (nullable = true)



In [ ]:
type(bilboard_df1_spark)

In [ ]:
type(bilboard_df1)

##### Get first 5 lines with array

In [ ]:
bilboard_df1_spark.take(5)

##### Format to show better

In [ ]:
display(bilboard_df1_spark.show(5))

In [ ]:
bilboard_df1_spark.count()

In [295]:
bilboard_df2_spark.count()

100

##### Turn False inferSchema

In [ ]:
bilboard_df1_spark = spark.read.format("csv").option("inferSchema", "False").option("header", "True").csv(path)
bilboard_df1_spark.collect()

##### Pyspark code

In [ ]:
from pyspark.sql.functions import *

bilboard_df1_spark.orderBy(expr("peak desc")).show(15)


In [ ]:
bilboard_df1_spark.orderBy(expr("artist asc")).show(15)

In [ ]:
display(bilboard_df1_spark.select("artist").show(10))

#### Using filter + order by

In [ ]:
display(bilboard_df1_spark.filter("this_week_chart > 4").orderBy(expr("title asc")).show(10))

##### Rename column in data frame

In [ ]:
bilboard_df1_spark = bilboard_df1_spark.withColumnRenamed("artist", "Artista").withColumnRenamed("title", "Musica").withColumnRenamed(
    "this_week_chart", "TocadasNaSemana").withColumnRenamed("last_week_chart", "TocadasSemanaPassada").withColumnRenamed("peak", "Pico").withColumnRenamed("weeks_on_chart", "GraficoSemanal")
bilboard_df1_spark.show()


##### ADD Column in dataFrame

In [297]:
bilboard_df1_spark = bilboard_df1_spark.withColumn("product", bilboard_df1_spark['peak']*100).show(10)

AnalysisException: Resolved attribute(s) peak#3378 missing from Musica#3401,Artista#3394,TocadasNaSemana#3408,TocadasSemanaPassada#3415,Pico#3422,GraficoSemanal#3429 in operator !Project [Musica#3401, Artista#3394, TocadasNaSemana#3408, TocadasSemanaPassada#3415, Pico#3422, GraficoSemanal#3429, (peak#3378 * 100) AS product#3481].;
!Project [Musica#3401, Artista#3394, TocadasNaSemana#3408, TocadasSemanaPassada#3415, Pico#3422, GraficoSemanal#3429, (peak#3378 * 100) AS product#3481]
+- Project [Musica#3401, Artista#3394, TocadasNaSemana#3408, TocadasSemanaPassada#3415, Pico#3422, weeks_on_chart#3344 AS GraficoSemanal#3429]
   +- Project [Musica#3401, Artista#3394, TocadasNaSemana#3408, TocadasSemanaPassada#3415, peak#3343 AS Pico#3422, weeks_on_chart#3344]
      +- Project [Musica#3401, Artista#3394, TocadasNaSemana#3408, last_week_chart#3342 AS TocadasSemanaPassada#3415, peak#3343, weeks_on_chart#3344]
         +- Project [Musica#3401, Artista#3394, this_week_chart#3341 AS TocadasNaSema

##### Drop column

In [ ]:
#bilboard_df1_spark = bilboard_df1_spark.drop("Valor").show(10)

In [ ]:
display(bilboard_df1_spark.show(50))